In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub # can pip install

In [2]:
#data sugmentation
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
training_set = train_datagen.flow_from_directory('image_data/training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 198 images belonging to 2 classes.


In [3]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('image_data/validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 100 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import MobileNet
from keras.layers import Dense, Input,TimeDistributed, Concatenate,Dense,Bidirectional,Flatten,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.applications import DenseNet121
from keras import Model,Sequential

from keras.layers.recurrent import LSTM

## CNN with RNN Combaine

In [7]:
#CNN

model_1 = Sequential()
model_1.add(Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu',input_shape=[64, 64, 3]))
model_1.add(Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_1.add(Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu'))
model_1.add(Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model_1.add(Flatten())
model_1.add(Dense(9 , activation='sigmoid'))

In [8]:
x =model_1.layers[5].output

x = TimeDistributed(Flatten())(x)

# # rnn model
# # x = Bidirectional(LSTM(256,dropout=0.5))(x)


x = LSTM(256,dropout=0.5)(x)


# #stacked rnn
# # x = LSTM(256,return_sequences=True ,dropout=0.5)(x)
# # x = LSTM(156,dropout=0.5)(x)

predictions = Dense(1, activation="sigmoid")(x)
my_model = Model(inputs=model_1.inputs, outputs=predictions)



In [9]:
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2_input (InputLayer)  [(None, 64, 64, 3)]      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        9248      
                                                             

## Transfer Learning with RNN

In [8]:
# cnn transfer learning model
base_model = MobileNet(include_top=False, weights='imagenet',input_shape= (224,224,3))
x = base_model.output

x = TimeDistributed(Flatten())(x)

# # rnn model
# # x = Bidirectional(LSTM(256,dropout=0.5))(x)


x = LSTM(256,dropout=0.5)(x)


# #stacked rnn
# # x = LSTM(256,return_sequences=True ,dropout=0.5)(x)
# # x = LSTM(156,dropout=0.5)(x)

predictions = Dense(1, name="dense", activation="sigmoid")(x)
my_model = Model(inputs=base_model.inputs, outputs=predictions)

In [9]:
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [10]:
my_model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [11]:
history = my_model.fit(
    training_set,
    epochs=1,
    validation_data=test_set
    )

7/7 [==============================] - 74s 9s/step - loss: 0.6643 - accuracy: 0.6414 - val_loss: 0.9452 - val_accuracy: 0.5000
